In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

import sys
sys.path.append('../')

from wavy import panel
from wavy import side
from wavy import block
from wavy import nn
from wavy import plot

In [2]:
df = pd.read_pickle('processed.pkl')

In [3]:
df.head(5)

LNC                                                    MAS  \
                 Open       High        Low      Close     Volume       Open   
Date                                                                           
2005-12-21  38.127885  39.324626  38.120589  39.207870  2190100.0  18.991695   
2005-12-22  39.185984  39.324630  38.879500  39.040039  1095500.0  19.085703   
2005-12-23  39.061931  39.397604  38.959771  39.324631   588800.0  18.954090   
2005-12-27  39.514374  39.514374  38.981676  39.237080   932700.0  18.929008   
2005-12-28  39.455973  39.696782  39.244353  39.310028   427200.0  18.853798   

                                                        ...       CSX  \
                 High        Low      Close     Volume  ...      Open   
Date                                                    ...             
2005-12-21  19.154659  18.903944  19.085712  1384200.0  ...  6.245725   
2005-12-22  19.298810  18.910202  18.929007  1576900.0  ...  6.381336   
2005-12-23  18.954090  18.684571  18.816196  2090500.0  ...  6.297691   
2005-12-27  18.991687  18.803650  18.847525  1609000.0  ...  6.520750   
2005-12-28  18.935281  18.747245  18.803656  2026300.0  ...  6.419358   

                                                            CI             \
                High       Low     Close     Volume       Open       High   
Date                                                                        
2005-12-21  6.399080  6.245725  6.362326  8358000.0  36.945102  37.462575   
2005-12-22  6.420626  6.361058  6.410487  5344800.0  37.409841  37.630677   
2005-12-23  6.520752  6.297691  6.468790  3630000.0  37.376876  37.432909   
2005-12-27  6.527087  6.369931  6.400348  4905000.0  37.195584  37.455973   
2005-12-28  6.459914  6.363592  6.419358  3578400.0  37.241735  37.449383   

                                             
                  Low      Close     Volume  
Date                                         
2005-12-21  36.895658  37.370289  2805900.0  
2005-12-22  37.073647  37.327442  2296800.0  
2005-12-23  36.905546  37.261517  1686000.0  
2005-12-27  36.961570  37.116482  1917900.0  
2005-12-28  37.109892  37.251621  1248900.0  

[5 rows x 2085 columns]

In [4]:
panel_ = panel.from_data(df, lookback = 5, horizon = 3, gap = 2, assets = ['AAPL', 'MSFT'], channels= ['High', 'Low'])

In [6]:
panel_

size                            3740
lookback                           5
horizon                            1
gap                                2
num_xassets                        2
num_yassets                        2
num_xchannels                      2
num_ychannels                      2
start            2005-12-21 00:00:00
end              2020-11-10 00:00:00
Name: Panel, dtype: object


<Panel, size 3740>

# Graph types

Separate X from Y

In [10]:
panel_.plot(2)

In [11]:
panel_.plot(3)

In [ ]:
panel_.x.first

In [ ]:
panel_.x[0]

In [ ]:
panel.x[0].index

In [12]:
panel_.x.plot(2)

In [ ]:
panel.x[0]

In [14]:
panel_.x[2].plot()

In [ ]:
panel.y[0]

In [ ]:
panel.set_training_split()

In [ ]:
panel.x.first

In [ ]:
panel.shape

# Baseline

In [ ]:
baseline = nn.Baseline(panel, metrics=['SE', 'MSE', 'RMSE'])

In [ ]:
predicted = baseline.predict()

In [ ]:
panel.test.y[0]

In [ ]:
predicted[2]

In [ ]:
predicted_val = baseline.predict_val()

In [ ]:
panel.val.y[0]

In [ ]:
predicted_val[2]

In [ ]:
baseline.evaluate('test')

In [ ]:
# m = tf.keras.metrics.MeanSquaredError()
# m.update_state(panel.test.y.tensor3d, predicted.tensor3d)
# m.result().numpy()

# DenseModel

In [ ]:
densemodel = nn.DenseModel(panel, model_type="regression")

In [ ]:
densemodel.fit()

In [ ]:
densemodel.model.summary()

In [ ]:
predicted = densemodel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
densemodel.evaluate('test')

In [ ]:
import numpy as np
import pandas as pd
import plotly as px
import plotly.graph_objects as go
import plotly.express as px

pd.set_option("multi_sparse", True)  # To see multilevel indexes
pd.options.plotting.backend = "plotly"

from plotly.subplots import make_subplots

def pair_plot(panel, idx):

    cmap = px.colors.qualitative.Plotly

    fig = make_subplots(rows=len(panel.channels), cols=len(panel.assets), subplot_titles=panel.assets, shared_xaxes=True)

    for j, channel in enumerate(panel.channels):
        c = cmap[j]
        for i, asset in enumerate(panel.assets):

            showlegend = i <= 0
            x_df = panel.x[idx].filter(assets=asset, channels=channel)
            y_df = panel.y[idx].filter(assets=asset, channels=channel)
            # y_df = panel.y[idx].filter(assets=asset, channels=channel)

            x_trace = go.Scatter(x=x_df.index, y=x_df.values.flatten(),
                            line=dict(width=2, color=c), showlegend=showlegend, name=channel)

            y_trace = go.Scatter(x=y_df.index, y=y_df.values.flatten(),
                                line=dict(width=2, dash='dot', color=c), showlegend=False)

            fig.add_trace(x_trace, row=j+1, col=i+1)
            fig.add_trace(y_trace, row=j+1, col=i+1)

    fig.update_layout(showlegend=True)
    fig.show()

In [ ]:
panel.x.as_dataframe()

In [ ]:
# panel = panel.filter('AAPL', ['High', 'Low'])
pair_plot(panel, 0)

In [ ]:
panel = panel.filter('LNC', ['Open', 'High'])
panel[0].plot()
panel.y_ = model.predict()

# ConvModel

In [ ]:
convmodel = nn.ConvModel(panel, model_type="regression")

In [ ]:
convmodel.fit()

In [ ]:
convmodel.model.summary()

In [ ]:
predicted = convmodel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
convmodel.evaluate()

# LinearRegression

In [ ]:
linearRegression = nn.LinearRegression(panel)

In [ ]:
linearRegression.fit()

In [ ]:
predicted = linearRegression.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
linearRegression.evaluate()

# Teste SeparateAssetModel

In [ ]:
panel.val.x[0]

In [ ]:
test_splits = panel.val.x._split_assets()

In [ ]:
x_train = [x.tensor3d for x in test_splits]

In [ ]:
# pd.DataFrame(x_train)

In [ ]:
test_splits[0].tensor3d[0]

In [ ]:
test_splits[0].tensor4d[0]

In [ ]:
pd.DataFrame(panel.val.x.tensor4d)

# SeparateAssetModel

In [ ]:
separateAssetModel = nn.SeparateAssetModel(panel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

In [ ]:
separateAssetModel.fit()

In [ ]:
separateAssetModel.model.summary()

In [ ]:
predicted = separateAssetModel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
separateAssetModel.evaluate()

# SeparateAssetConvModel

In [ ]:
separateAssetConvModel = nn.SeparateAssetConvModel(panel, loss='MSE', optimizer='adam', metrics=['MAE'], hidden_activation='relu', last_activation='linear')

In [ ]:
separateAssetConvModel.fit()

In [ ]:
separateAssetConvModel.model.summary()

In [ ]:
predicted = separateAssetConvModel.predict()

In [ ]:
predicted[0]

In [ ]:
panel.test.y[0]

In [ ]:
separateAssetConvModel.evaluate()

# Test Docstrings

In [ ]:
densemodel = nn.DenseModel()